<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/QuestionAugmentedSquaD_2000examples_FlatDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test Settings:

1. **Model**: MT5-base;
2. **Dataset**: LMQG/QG-Squad;
3. **Examples**: $10000$;
4. **Other**: $2000$ from training set augmented to $10000$ with question augmentation (one example per pair $\langle$ answer, context, question $\rangle$)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [6]:
%run "AveragedCrossEntropy.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.9 MB/s eta

In [7]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os
from dotenv import load_dotenv

In [8]:
model_checkpoint='google/mt5-base'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
tokenizer.add_special_tokens({'sep_token': '<sep>', 'additional_special_tokens': ['<answer>', '<context>', '<hl>']})

4

In [11]:
MAX_INPUTS_LENGTH = 512
MAX_LABELS_LENGTH = 64
def tokenizer_function(
    samples, max_inputs_length=MAX_INPUTS_LENGTH, max_labels_length=MAX_LABELS_LENGTH,
    input_ids_padding=True, train_dataset=None, ignore_index_id=0, question_column='question',
):
  max_labels_length = max_labels_length if not train_dataset else get_maximum_labels_length(train_dataset)
  input_tokenized = tokenizer(samples['answer_context'], padding=input_ids_padding, max_length=max_inputs_length, truncation=True, return_tensors='pt')
  labels_tokenized = tokenizer(samples[question_column], padding="max_length", max_length=max_labels_length, truncation=True, return_tensors='pt')
  labels, masks = labels_tokenized['input_ids'], labels_tokenized['attention_mask']
  argmin_masks = torch.argmin(masks, dim=-1)
  for index in range(len(argmin_masks)):
    if masks[index][argmin_masks[index]] == 0 and ignore_index_id != 0:
      labels[index][argmin_masks[index]:] = ignore_index_id
  input_tokenized['labels'] = [label for label in labels]
  return input_tokenized

In [12]:
def get_maximum_inputs_length(dataset):
  tokenized_dataset_lengths = [len(tokenizer.tokenize(sample['answer_context'])) for sample in dataset]
  return max(tokenized_dataset_lengths)

In [13]:
def load_and_preprocess_question_augmented_squad(
  dataset_name='lmqg/qg_squad', train_dataset_name='lmqg_squad_augmented_train',
  eval_dataset_name='lmqg_squad_augmented_eval', test_dataset_name='lmqg_squad_augmented_test',
  train_select=None, eval_select=None, use_extra_ids=False, num_questions_per_example=5,
):
  dataset_loading_result = get_dataset(dataset_name, train_dataset_name, eval_dataset_name, test_dataset_name)
  local = dataset_loading_result['local']
  if local:
    train_dataset = dataset_loading_result['train']
    validation_dataset = dataset_loading_result['eval']
    test_dataset = dataset_loading_result['test']
  if not local:
    datasets = load_dataset(dataset_name)
    train_dataset = datasets['train'].remove_columns(['question'])
    validation_dataset = datasets['validation'].remove_columns(['question'])
    test_dataset = datasets['test'].remove_columns(['question'])
    print(f"Train dataset has {len(train_dataset)} items. Validation dataset has {len(validation_dataset)} items.")

    train_dataset.save_to_disk(train_dataset_name)
    validation_dataset.save_to_disk(eval_dataset_name)
    test_dataset.save_to_disk(test_dataset_name)

  if train_select:
    train_dataset = train_dataset.select(range(train_select))
  if eval_select:
    validation_dataset = validation_dataset.select(range(eval_select))

  print(train_dataset)
  answers, contexts, questions, question_repeated = [], [], [], []
  num_questions_to_take = min(len(train_dataset['questions'][0]), num_questions_per_example)
  for item in train_dataset:
    for i, question in enumerate(item['questions']):
      if i >= num_questions_to_take:
        break
      question_repeated.append(item['question'])
      answers.append(item['answer'])
      contexts.append(item['context'])
      questions.append(question)
  print(len(answers), len(questions), len(contexts))
  train_dataset = Dataset.from_dict({'answer': answers, 'questions': questions, 'context': contexts, 'question': question_repeated})

  build_train_feature_local = lambda sample: build_train_feature(sample, use_extra_ids=use_extra_ids, context_label='context')
  train_dataset = train_dataset.map(build_train_feature_local).remove_columns(['answer', 'context'])
  validation_dataset = validation_dataset.map(build_train_feature_local).remove_columns(['answer', 'context'])
  test_dataset = test_dataset.map(build_train_feature_local).remove_columns(['answer', 'context'])

  max_train_inputs_length = get_maximum_inputs_length(train_dataset)
  max_train_labels_length = get_maximum_labels_length(train_dataset)
  print(max_train_inputs_length, max_train_labels_length)
  train_tokenizer = lambda samples: tokenizer_function(samples, question_column='questions', input_ids_padding='max_length', max_inputs_length=max_train_inputs_length, max_labels_length=max_train_labels_length)
  tokenized_train_dataset = train_dataset.map(train_tokenizer, batched=False).remove_columns(['answer_context', 'question', 'questions'])
  tokenized_validation_dataset = validation_dataset.map(tokenizer_function, batched=True).remove_columns(['answer_context', 'question'])
  tokenized_test_dataset = test_dataset.map(lambda samples: tokenizer_function(samples, input_ids_padding="max_length", max_labels_length=32), batched=True).remove_columns(['answer_context', 'question'])

  tokenized_train_dataset = tokenized_train_dataset.map(lambda sample: {'input_ids': sample['input_ids'][0], 'attention_mask': sample['attention_mask'][0], 'labels': sample['labels'][0]})
  tokenized_train_dataset.set_format("torch")
  tokenized_validation_dataset.set_format("torch")
  tokenized_test_dataset.set_format("torch")

  return (train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset)

In [14]:
(train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset) = \
  load_and_preprocess_question_augmented_squad(train_dataset_name='lmqg_squad_augmented_train_0_2000', num_questions_per_example=5, eval_select=2000)

Dataset({
    features: ['answer', 'question', 'context', 'questions'],
    num_rows: 2000
})
10000 10000 10000


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

683 44


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

### 3. Model Loading and configuration

#### Loading

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 250104. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(250104, 768)

In [ ]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250104, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250104, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

#### Configuration

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
NUM_EPOCHS = 10

In [ ]:
optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
  num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-4, num_epochs=NUM_EPOCHS)

50000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 4. Fine-tuning

#### Execution

In [ ]:
tokenized_train_dataset['labels'].shape

torch.Size([10000, 44])

In [ ]:
training_results_dict = main_training_loop(
    model, device, optimizer, train_dataloader, eval_dataloader,
    lr_scheduler, num_training_steps, num_epochs=NUM_EPOCHS, metrics=None,
    eval_strategy='epoch', eval_every=2000, model_save_path='lmqg_squad_augmented_mt5base_20epochs_2000examples',
    early_stopping=True, early_stopping_patience=10, tokenizer=tokenizer,
  )

epoch_train_losses = training_results_dict['epoch_train_losses']
epoch_eval_losses = training_results_dict['epoch_eval_losses']
epoch_eval_metrics = training_results_dict['epoch_eval_metrics']
print(epoch_train_losses, epoch_eval_losses, epoch_eval_metrics, sep='\n')

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 0: Train Loss = 2.764678716659546, Eval Loss = 2.371234893798828
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 1: Train Loss = 1.7972885370254517, Eval Loss = 1.9653733968734741
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 2: Train Loss = 1.0632888078689575, Eval Loss = 1.5438541173934937
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 3: Train Loss = 0.5043489933013916, Eval Loss = 0.34864509105682373
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 4: Train Loss = 0.178693488240242, Eval Loss = 0.2782707214355469
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 5: Train Loss = 0.31684792041778564, Eval Loss = 0.29162293672561646
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 6: Train Loss = 0.3581695854663849, Eval Loss = 0.3061820864677429
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 7: Train Loss = 0.40888357162475586, Eval Loss = 0.31296879053115845
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 8: Train Loss = 0.38944295048713684, Eval Loss = 0.31036922335624695
Save this model (y/n)?> n
Continue training (y/n)?> n
[2.764678716659546, 1.7972885370254517, 1.0632888078689575, 0.5043489933013916, 0.178693488240242, 0.31684792041778564, 0.3581695854663849, 0.40888357162475586, 0.38944295048713684]
[2.371234893798828, 1.9653733968734741, 1.5438541173934937, 0.34864509105682373, 0.2782707214355469, 0.29162293672561646, 0.3061820864677429, 0.31296879053115845, 0.31036922335624695]
[]


In [ ]:
model.save_pretrained(f"squad_qg_augmented_allitems_mt5base_20epochsmax_2000examples_(epoch {training_results_dict['epoch']})")

In [ ]:
from time import sleep
sleep(180)

### 5. Analysis of the Results

### Calculating BLEU and ROUGE score

In [ ]:
loss_tracker=[]
metrics_tracker=[]
metrics = {
    'bleu': get_bleu_config(tokenizer),
    'nist': get_nist_config(tokenizer),
    'rouge': get_rouge_config(tokenizer),
}

In [16]:
P = True

In [17]:
if P:
  model = MT5ForConditionalGeneration.from_pretrained('squad_qg_augmented_mt5base_2000examples_epoch9', local_files_only=True)

In [18]:
if P:
  optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
    num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-4, num_epochs=2)

5000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
if P:
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)
  print()

In [20]:
test_dataset, tokenized_test_dataset = test_dataset.shuffle(seed=42), tokenized_test_dataset.shuffle(seed=42)

In [21]:
tokenized_test_dataset_reduced = tokenized_test_dataset.select(range(200))
test_dataset_reduced = test_dataset.select(range(200))

In [ ]:
test_dataset['question'][16:18]

["Le grand concludes that an author's words offer only what they intended for them to imply regarding this type of terminology?",
 'Khitan had been overthrown by which dynasty that Genghis Khan later defeated?']

In [ ]:
final_predictions = select_best_output(
    model, tokenizer, tokenized_test_dataset['input_ids'][16:18].to(device), test_dataset[16:18]['question'], score_function=bertscore_f1based_score,
    max_length=64, num_beams=4, top_k=None, top_p=None, num_candidates=4, verbose=False, tokenize_output=False
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
final_predictions

['What is the significance of specific terminology?',
 'What dynasty did Genghis Khan invite a Khitan prince?']

In [ ]:
from tqdm.auto import tqdm
test_dataloader = DataLoader(tokenized_test_dataset, shuffle=True, batch_size=8, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
test_loss_tracker, test_metrics_tracker, num_test_steps = [], [], len(test_dataloader)
test_progress_bar = tqdm(range(num_test_steps))
test_loss = evaluation_loop(
    model, device, optimizer, test_dataloader, lr_scheduler, test_loss_tracker, test_metrics_tracker, metrics, test_progress_bar,
    tokenizer=tokenizer, num_beams=4, num_candidates=4, tokenize_predictions_output=False,
)

  0%|          | 0/1322 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sa

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Metrics = {'bleu': {'bleu': 0.19702508079891198, 'precisions': [0.4884865313365153, 0.23393938303480663, 0.14342807718691283, 0.09193818945310331], 'brevity_penalty': 1.0, 'length_ratio': 1.0159494558322761, 'translation_length': 121727, 'reference_length': 119816}, 'nist': {'nist_mt': 5.213706552330381}, 'rouge': {'rouge1': 0.4842570182006559, 'rouge2': 0.2600564617600638, 'rougeL': 0.4479914385162102, 'rougeLsum': 0.4479831513431553}}


In [22]:
bertscore = compute_bert_score(
    test_dataset, tokenized_test_dataset, model, device, tokenizer, batch_size=16,
    lang='en', model_type=None, max_length=100, num_beams=4, num_candidates=1
)

  0%|          | 0/661 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
bertscore

{'precision': (0.9112017396414784, 0.009016935719912309),
 'recall': (0.9091537364356999, 0.009173696305815039),
 'f1': (0.9099984976734949, 0.008563319336730608)}

In [24]:
np.mean([v[0] for v in bertscore.values()])

0.9101179912502243